In [15]:
import pandas as pd
import numpy as np

In [16]:
cars_clean = pd.read_csv('cars_clean.csv')

In [17]:
cars_clean

,symboling,normalized_losses,num_of_doors,wheel_base,length,width,height,curb_weight,num_of_cylinders,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,1.725050,0.000000,-1.156378,-1.685107,-0.439409,-0.853460,-2.034081,-0.014858,-0.343660,0.075389,0.520894,-1.829927,-0.291435,0.203984,-0.246556,-0.652249,-0.542288,13495.0
1,1.725050,0.000000,-1.156378,-1.685107,-0.439409,-0.853460,-2.034081,-0.014858,-0.343660,0.075389,0.520894,-1.829927,-0.291435,0.203984,-0.246556,-0.652249,-0.542288,16500.0
2,0.127193,0.000000,-1.156378,-0.710103,-0.244152,-0.185597,-0.559713,0.518080,1.548823,0.606234,-2.433435,0.675938,-0.291435,1.357649,-0.246556,-0.964397,-0.689386,16500.0
3,0.926121,1.315931,0.864769,0.165748,0.195176,0.148335,0.218425,-0.423766,-0.343660,-0.431327,-0.526210,0.453899,-0.041121,-0.037480,0.801833,-0.184027,-0.100993,13950.0
4,0.926121,1.315931,0.864769,0.099646,0.195176,0.243744,0.218425,0.520017,0.602582,0.220165,-0.526210,0.453899,-0.541748,0.311302,0.801833,-1.120471,-1.277779,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,-1.470664,-0.845956,0.864769,1.702619,1.187733,1.436357,0.709881,0.768075,-0.343660,0.340812,1.680188,-0.339096,-0.166278,0.284473,0.592155,-0.340101,-0.395189,16845.0
197,-1.470664,-0.845956,0.864769,1.702619,1.187733,1.388653,0.709881,0.956057,-0.343660,0.340812,1.680188,-0.339096,-0.366529,1.518625,0.382477,-0.964397,-0.836484,19045.0
198,-1.470664,-0.845956,0.864769,1.702619,1.187733,1.436357,0.709881,0.884353,1.548823,1.112950,0.932257,-1.227251,-0.341497,0.821061,0.801833,-1.120471,-1.130681,21485.0
199,-1.470664,-0.845956,0.864769,1.702619,1.187733,1.436357,0.709881,1.281633,1.548823,0.437329,-1.199348,0.453899,3.212954,0.069837,-0.665911,0.128120,-0.542288,22470.0


In [18]:
# Normalize features
cars_features = cars_clean.drop('price', axis = 1)
feature_cols = cars_features.columns
feature_cols

Index(['symboling', 'normalized_losses', 'num_of_doors', 'wheel_base',
       'length', 'width', 'height', 'curb_weight', 'num_of_cylinders',
       'engine_size', 'bore', 'stroke', 'compression_ratio', 'horsepower',
       'peak_rpm', 'city_mpg', 'highway_mpg'],
      dtype='object')

In [19]:
# Import model & validation methods from sklearn 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Training & validation function
def knn_train_test(feature_col, target_col, df):
    train, test = train_test_split(df, train_size = 0.8, test_size = 0.2, random_state = 1)
    model = KNeighborsRegressor()
    model.fit(train[feature_col], train[target_col])
    predictions = model.predict(test[feature_col])
    mse = mean_squared_error(test[target_col], predictions)
    rmse = np.sqrt(np.abs(mse))
    return rmse

In [56]:
def forward_selection_alt(features, target, df, min_rmse, results):
    all_features = df.drop(target,axis=1).columns
    updated_features = [feature for feature in all_features if feature not in features]
    l = len(updated_features)
    
    if l > 0:       
         # Initialize a dictionary with feature combos and their rmse score   
        feature_rmse = dict()
        
        for i in range(0, l):
            # Get the base features for combo
            combo = [feature for feature in features]
            
            # Append each feature in updated_fatures to combo
            combo.append(updated_features[i])
            
            # Assign rmse of the combo to feature_rmse dictionary 
            rmse = knn_train_test(combo, target, df)
            feature_rmse[tuple(combo)] = rmse
        
        # Get the item with the minmum value in feature_rmse dictionary 
        min_feature_rmse = min(feature_rmse.items(), 
                               key = lambda item: item[1])
        
        min_combo_rmse = min_feature_rmse[1]
        min_combo = min_feature_rmse[0]
#         print('batch best features: ', min_combo)
        print('batch minimum rmse: ', min_combo_rmse, ' | ',
              'true minimum: ', min_rmse)
        print('\n')

        results[min_combo] = min_combo_rmse
        
        if  min_combo_rmse < min_rmse:
            min_rmse = min_combo_rmse
                
        # Return the key of the minmum value in feature_rmse dictionary as the new features list 
        features = list(min_combo)
                
        # Rinse & repeat  
        forward_selection_alt(features, target, df, min_rmse, results)
    
    best_result = min(results.items(),
                     key = lambda item: item[1])

    return (list(best_result[0]), best_result[1]) 

In [57]:
forward_selection_alt([], 'price', cars_clean, 2700, {})

batch minimum rmse:  3228.411945485907  |  true minimum:  2700


batch minimum rmse:  2359.5356909937113  |  true minimum:  2700


batch minimum rmse:  2141.543920610184  |  true minimum:  2359.5356909937113


batch minimum rmse:  2019.205723823591  |  true minimum:  2141.543920610184


batch minimum rmse:  2083.5605151969557  |  true minimum:  2019.205723823591


batch minimum rmse:  2013.6462423346093  |  true minimum:  2019.205723823591


batch minimum rmse:  2119.4119147514116  |  true minimum:  2013.6462423346093


batch minimum rmse:  2266.2327669827614  |  true minimum:  2013.6462423346093


batch minimum rmse:  2234.3188642888626  |  true minimum:  2013.6462423346093


batch minimum rmse:  2303.8158685315025  |  true minimum:  2013.6462423346093


batch minimum rmse:  2273.0146053418925  |  true minimum:  2013.6462423346093


batch minimum rmse:  2305.4330540766664  |  true minimum:  2013.6462423346093


batch minimum rmse:  2438.994841142955  |  true minimum:  2013.64624233460

(['horsepower', 'curb_weight', 'bore', 'width', 'engine_size', 'city_mpg'],
 2013.6462423346093)

In [31]:
def forward_selection(features, target, df, min_rmse):
    """
    Trains a KNN model following the train/test validation procedure selecting features iteratively
    Starts with empty set and adds progressively new features following the RMSE criterion.
    
    features (list of string): the training features
    target (string): the column target
    df (DataFrame): the data
    min_rmse (float): the minimum RSME value we want to minimize    
    
    outputs :
        RMSE (float)
        featurees (list of string): the best feature set found by the algorithm
    """    
    all_features = df.drop(target,axis=1, inplace = False).columns
    l=len(all_features)-len(features)
    updated_features = [feature for feature in all_features if feature not in features]
    selected_features = []
    for i in range(0,l): 
        features.append(updated_features[i])        
        avg_rmse = knn_train_test(features, target, df)
        if avg_rmse < min_rmse:
            min_rmse = avg_rmse            
            selected_features.append(updated_features[i])
            features.remove(updated_features[i])
        else:
            features.remove(updated_features[i])
    if len(selected_features)>0:
        features.append(selected_features[-1])       
        forward_selection(features, target, df, min_rmse)
       
    return(knn_train_test(features, target, df), features)

*min_rmse value restriction in the function as shown below*:

In [52]:
forward_selection([], 'price', cars_clean, 4000)

(2019.205723823591, ['horsepower', 'curb_weight', 'bore', 'width'])

In [51]:
forward_selection([], 'price', cars_clean, 2700)

ValueError: at least one array or dtype is required